In [2]:
### All Imports go here

import os
import sys
import boto
from inspect import getmembers
from datetime import datetime
HOME = os.environ.get('HOME')
sys.path.append(HOME + "/comscore_processing")
sc.addPyFile(HOME + "/comscore_processing/utils.zip")
from utils.utils import apply_process_row
from utils.constants import *
from utils.session_aggregators import SessionAggregator
from utils.log_processors import UserProcessor
from utils.category_counts import site_combos_to_process,process_category_combos
from utils.page_site_counter import PageSiteCounter

In [2]:
### Data Declarations go here

data = sc.textFile("/home/analytics/comscore_processing/tests/TestSpark/sample_data/20160404")
header = sc.broadcast(data.first().strip().split('\t'))
sampleData = data.sample(False,0.01)

In [3]:
S3_FILE_PATTERN.format('20160404')

's3n://msr-redshift/comscore_ctu_dump_url_dat_20160404_RDNA_*.dat.gz'

In [ ]:
### All constants for data input and output

### Input format - Text file
S3_FILE_PATTERN.format('20160404')

### Output Pattern
### Data dhould have already been processed to get the data from the buckets

## Visitor level processing / First pass pattern - Parquet format data
S3_FIRST_PASS_PATTERN.format(log_date='2016-03-01',site='tasteofhome')

## Site and page counting / daily page view/site view counts - Parquet format data
S3_PAGE_COUNT_PATTERN.format(log_date="2016-04-05")
S3_SITE_COUNT_PATTERN.format(log_date="2016-04-05")

## gives output of all vids and sites visited by them. one vis and site per row - Parquet data
##[Row(ns_vid='1d1d68d4-6903-1097-0113-145934844600', log_date='2016-04-06', site='tasteofhome'),
## Row(ns_vid='aaa7ae17-2024-7090-1251-458873183000', log_date='2016-04-06', site='tasteofhome')]
userIdCountData = sqlContext.read.\
                        parquet("s3n://rdmsr-anl/daily_all_ids_by_site/2016-04-06/log_date=2016-04-06/site=tasteofhome/")
userIdCountData.take(10)

## Final pass pattern - First pass pattern data should exist to process final pass - text file
S3_VISITOR_FINAL_PATTERN.format(site="tasteofhome",start="2016-03-01",end="2016-03-31",source="parquet")

## Category Counting - Parquet data
S3_SITE_CAT_FILE_PATTERN % ("2015-12-01","_site-category-subcat")

In [8]:
### Data of S3_FILE_PATTERN
data_S3FilePattern = sc.textFile(S3_FILE_PATTERN.format('20160710'))
#data_S3FilePattern.take(2)
sampIpFilePat = data_S3FilePattern.sample(False,0.01)
header = sc.broadcast(data_S3FilePattern.first().strip().split('\t'))
header.value

['ns_vid',
 'ns_utc',
 'name',
 'ns_lda_mark_reason',
 'referrer',
 'ns_v_country',
 'ns_v_city',
 'content_id',
 'ns_src_site',
 'ns_jspageurl',
 'ns_site',
 '_site',
 'category',
 'subcat',
 'subsubcat',
 'subsubsubcat',
 'content_type',
 'content_name',
 'ns_campaign',
 'epid',
 'internal_tracking_id',
 'ns_search_term',
 'ns_search_result',
 'search_pg',
 'search_pg_no',
 'search_cat',
 'search_pos',
 'search_event',
 'optin',
 'optout',
 'nl_name',
 'nl_signup_event',
 'pv_event',
 'mpv_event',
 'tellafriend_event',
 'sweeps_entry_event',
 'link_pos',
 'link_page',
 'link_name',
 'exit_link',
 'print_event',
 'form_type',
 'form_name',
 'form_sub_event',
 'slideshow_event',
 'exit_link_event',
 'comments_event',
 'payment_type',
 'is_mobile',
 'login_status',
 'rel_content',
 'search_facet_term',
 'search_facet_remove',
 'form_sub_pref',
 'page_no',
 'partner_name',
 'login_type',
 'member_id',
 'order_source',
 'reg_source',
 'mailing_id',
 'dir_type',
 'usr_agent',
 'agent',
 'l

In [6]:
### First pass pattern - Output of Write parquet
data_S3FirstPass = sqlContext.read.parquet(S3_FIRST_PASS_PATTERN.format(log_date='2016-03-01',site='tasteofhome'))
data_S3FirstPass.take()[:1]

[Row(id='3fd45d12-2307-7017-1012-014201642930', ruid='dbb1641c-1420-4971-8b01-22315f912b1c', sessions=[{'user_agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/48.0.2564.116 Safari/537.36', 'records_in_session': '3', 'ip': 'dbb1641c-1420-4971-8b01-22315f912b1c', 'value_details': "{'content_id': {'30613': 2}, 'page_no': {'view all': 2}, 'partner_name': {'simple & delicious': 2}, 'type': {'hidden': 1, 'view': 2}, 'content_type': {'recipe detail': 2}, 'rel_content': {'related recipes': 2}}", 'categories_viewed': "[['recipes', 3]]", 'session_start_utc': '1456841130.16', 'epid': "['df5708cd5de442af52a21f5dbb50867cbc206ae0']", 'duration': '27', 'session_start_time': '2016-03-01 08:05:30', 'external_referrers': "['www.pinterest.com/']", 'time_zone': 'America/Chicago', 'location': "['41.7339', '-87.7795']", 'page_path': "[['www.tasteofhome.com/recipes/chinese-beef-casserole', 22]]"}], log_date='2016-03-01', site='tasteofhome')]

In [ ]:
### Page Count - Unique page URL and number of visits for that on that day
data_S3PageCount = sqlContext.read.parquet(S3_PAGE_COUNT_PATTERN.format(log_date="2016-04-05"))
data_S3PageCount.toPandas()

In [ ]:
### Site Count - Site and number of unique visitors
data_S3SiteCount = sqlContext.read.parquet(S3_SITE_COUNT_PATTERN.format(log_date="2016-04-05"))
data_S3SiteCount.toPandas()

In [ ]:
## gives output of all vids and sites visited by them. one vis and site per row - Parquet data
##[Row(ns_vid='1d1d68d4-6903-1097-0113-145934844600', log_date='2016-04-06', site='tasteofhome'),
## Row(ns_vid='aaa7ae17-2024-7090-1251-458873183000', log_date='2016-04-06', site='tasteofhome')]
userIdCountData = sqlContext.read.\
                        parquet("s3n://rdmsr-anl/daily_all_ids_by_site/2016-04-06/log_date=2016-04-06/site=tasteofhome/")
userIdCountData.take(10)

In [3]:
## Final pass pattern - First pass pattern data should exist to process final pass - text file
data_S3FinalPattern = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="tasteofhome",\
                                                                  start="2016-03-01",end="2016-03-31",source="parquet"))
data_S3FinalPattern.take(0)

['216.53.142.2#48C66211\ttasteofhome\t1\t0\t1.0\t5.0\t0\tMobile Safari\tbutter:1:5,easy:1:5,fudge:1:5,peanut:1:5,recipes:1:5\tno orders\tno payments\t2016-03-27\t3\t\tno ehid\tno email\tno epid\t1\t\t0\t\tRV=1/RE=1459080435/RB=/RO=8/RU=http%3a%2f%2fwww.tasteofhome.com%2frecipes%2feasy-peanut-butter-fudge/RS=^ADAMAxZXvrgGBPJP6n9z4Hm197Bmy4-\t\t0\taa8aec29-626f-4517-b5f4-22b501680615\t0\t1\t\t0\t\t1\t\t0\t1\t\t3\t\t\t1\tiOS\t1\t1\t\t0\t\t0\t\t\t0\t5\t0\t1\t0',
 '5a620820-2303-0105-0203-142417594900\ttasteofhome\t1\t1\t1.0\t5.0\t0\tChrome Mobile\tloaf:1:5,meat:1:5,mom:1:5,recipes:1:5\tno orders\tno payments\t2016-03-02\t28\t\tno ehid\tno email\tno epid\t1\t\t0\t\t\t\t0\t9c558b1f-927e-4801-a50c-5e8c36addc1c\t0\t1\t\t0\t\t1\t\t0\t1\t\t28\t\t\t1\tAndroid\t1\t1\t\t0\t\t0\t\t\t0\t5\t1\t0\t0']

In [5]:
### Headers explaining the above data
sa = SessionAggregator(datetime(2016, 3, 1), 
        datetime(2016, 3, 31))
header = ['visitor_id', 'site']
for member in getmembers(sa):
    if callable(member[1]) and not member[0].startswith('_'):
        header.append(member[0])
header = sc.broadcast(header)
header.value

['visitor_id',
 'site',
 'abandon_indicator',
 'afternoon_visit_indicator',
 'average_page_views_per_visit',
 'average_visit_time_per_category',
 'average_visit_time_per_day',
 'average_visit_time_spent',
 'bing_referral',
 'browser',
 'closed_hc_popup',
 'content_terms',
 'date_of_last_order',
 'date_of_last_payment_event',
 'date_of_last_visit',
 'days_since_last_abandon',
 'days_since_last_customer_support_visit',
 'ehid',
 'email',
 'epid',
 'evening_visit_indicator',
 'facebook_content_terms',
 'facebook_referral',
 'first_external_referrer_domain_to_site',
 'first_internal_referrer_to_site',
 'google_plus_content_terms',
 'google_referrel',
 'healthcare_conditions_selected',
 'ip_address',
 'is_bot',
 'is_mobile',
 'list_of_newsletters',
 'lunch_visit_indicator',
 'mean_time_between_visits',
 'morning_visit_indicator',
 'most_frequent_referrer',
 'newsletter_signup_indicator',
 'night_owl_visit_indicator',
 'number_days_since_first_visit',
 'number_days_since_last_visit',
 'numbe

In [ ]:
### Map header and values of Final Pattern data

def getMappedData(x):
    splittedRow = x.split('\t')
    x = dict(zip(header.value[:len(splittedRow)], splittedRow))
    return x

def mapData(visitorFinalPassData):
    vData = visitorFinalPassData.map(getMappedData)
    return vData
finalMappedData = mapData(data_S3FinalPattern)
finalMappedData.take(2)

In [ ]:
# sends mail if log date data is not available.
# if available returns data and header
data,header = get_log_date('20160406',sc)

In [ ]:
# writes log date to dyname saying the dates are processed
conn = DynamoDBConnection()
record_log_date("2016-04-06","site_category_counts", conn = conn)

In [9]:
processed = apply_process_row(sampIpFilePat, header, 15)
processed.take(15)[:3]

[(('100.34.173.161#562B4D10', 'familyhandyman'),
  [{'_site': 'familyhandyman',
    '_sweepstakes_and_discounts_from_the_fmaily_handyman._signup_event': '',
    'adblock': '',
    'add_bookmark_event': '',
    'add_groc_event': '',
    'age_check': '',
    'agent': 'Mozilla/5.0 (iPad; CPU OS 9_3_2 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13F69 Safari/601.1',
    'autocomplete_event': '',
    'autocomplete_recipe_event': '',
    'back_to_school_signup_event': '',
    'birds__blooms_signup_event': '',
    'breadcrumb': '',
    'breadcrumb_level': '',
    'breadcrumb_name': '',
    'building_and_construction_professionals_tips__hints_signup_event': '',
    'campaign': '',
    'cancel_mag_event': '',
    'cancel_ser_event': '',
    'category': 'skills',
    'channel_level': '',
    'christmas_cookie_countdown_signup_event': '',
    'comments_event': '',
    'conditions_selected': '',
    'content_id': '185294',
    'content_name': '',
    'content_type': '

NameError: name 'processed' is not defined

In [19]:
## Visitor first pass processing through functions
#processed = apply_process_row(data_S3FilePattern, header, 15)
dt = datetime(2015,5,9)
up = UserProcessor(dt)
data = processed.mapValues(up).map(lambda x: x[1], True)
data.take(5)[1]
filtered_data = data.filter(lambda x: len(x[3])>1)
filtered_data.take(5)

[('675e8a33-1240-1060-1740-811435096939',
  'rd',
  '2015-05-09',
  [{'categories_shared_on_facebook': 'None',
    'categories_shared_on_google_plus': 'None',
    'categories_shared_on_pinterest': 'None',
    'categories_shared_on_twitter': 'None',
    'categories_viewed': [['culture', 5]],
    'category_page_path': [['culture',
      'www.rd.com/advice/work-career/how-to-look-smart/9/']],
    'city': 'makati city',
    'country': 'Philippines',
    'duration': 5,
    'event_details': {'pv_event': {'true': 1}, 'slideshow_event': {'true': 1}},
    'internal_referrers': ['www.rd.com/advice/work-career/how-to-look-smart/1/'],
    'ip': '112.205.193.50',
    'location': ['14.5833', '120.9667'],
    'page_path': [['www.rd.com/advice/work-career/how-to-look-smart/9/', 5]],
    'records_in_session': 1,
    'session_start_time': '2016-07-10 18:03:24',
    'session_start_utc': '1468145004.392',
    'time_zone': 'Asia/Manila',
    'user_agent': 'Mozilla/5.0 (Linux; Android 5.0; SM-N9005 Build/LR

In [7]:
### Category Counts - Checks if already processed and returns only non processed dates
### given a ragne of dates, last date is excluded : 15-18 means 15,16,17
dates = [datetime(2016,4,x) for x in range(15,18)]
#dates = [datetime(2016,4,7)]
cats = site_combos_to_process(dates)
process_category_combos(data_S3FilePattern, header, cats, sqlCtx)

In [ ]:
### Site and page counting

PageSiteCounter